<a href="https://colab.research.google.com/github/LACDR-CDS/SCDR_Bioinformatics_Practical/blob/main/Session2/Session2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 2

In this session, you will go through the basic processing steps of RNA sequencing data. Using the steps taught in the lecture and repeated below, as well as the **cheat sheet** provided on Brightspace/Teams, you will write your own code to process and analyze the data.

Each team receives a count matrix with 4 samples: **2 ESC replicates** and 2 replicates which belong to one of the following stages of cardiomyocyte differentiation:

*   mesoderm
*   cardiomyocyte progenitors
*   cardiomyocytes
*   fetal heart

Your goal is to **label the samples with the correct stage** (both ESCs and differentiated).



## Setup

Run the following cells to set up the necessary packages and download the data. If you wish to use a package which is not in the list below, you will need to install and import it yourself.

In [ ]:
#Install packages which are not in the default environment
%pip install scanpy
%pip install pydeseq2
%pip install gseapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 kB 13.2 MB/s eta 0:00:00


In [ ]:
#Import packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as ad
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import pickle
import gseapy


In [ ]:
group_number =

In [ ]:
#Make data directory if it does not exist
os.makedirs("data", exist_ok=True)
os.makedirs("plots", exist_ok=True)

#Download datasets in the data folder
!wget https://raw.githubusercontent.com/LACDR-CDS/SCDR_Bioinformatics_Practical/refs/heads/main/Session2/datasets/group{group_number}.txt -O data/group{group_number}.txt
!wget https://raw.githubusercontent.com/LACDR-CDS/SCDR_Bioinformatics_Practical/refs/heads/main/Session2/datasets/metadata.csv -O data/group{group_number}_metadata.csv

--2025-09-18 12:51:18--  https://raw.githubusercontent.com/LACDR-CDS/SCDR_RNAseq/refs/heads/main/data/session2/dataset8_group2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 893053 (872K) [text/plain]
Saving to: ‘data/group2.txt’

data/group2.txt     100%[===================>] 872.12K  --.-KB/s    in 0.03s   

2025-09-18 12:51:18 (31.7 MB/s) - ‘data/group2.txt’ saved [893053/893053]

--2025-09-18 12:51:18--  https://raw.githubusercontent.com/LACDR-CDS/SCDR_RNAseq/refs/heads/main/data/session2/metadata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting

## Data import

In the empty cell(s) below, please do the following and answer the given questions:

1) Read the count matrix (the path should be: `"data/groupN.txt`", where N is your group number).
2) Check the orientation of the count table (`genes x samples` or `samples x genes`).
3) How many samples and genes there are in the count table?
4) Read the corresponding metadata table ( `"data/groupN_metadata.csv`"). What cell line do the samples belong to?

## Filtering

Now that you have become familiar with the structure of your **raw data**, you can start the processing steps. First, the data must be filtered such that we remove **genes with less than 10 reads over all samples**. We do this to remove possible noise/experimental artefacts, which decrease statistical confidence. Reference the cheat sheet for help with coding. Perform the filtering and answer the following questions:

1) Which genes have low overall read counts (lowest total gene count number that is higher than 0)?

*Hint: use gene_count_sums.sort_values() to sort the results*

2) How many genes are left in the count table after filtering?
3) How many reads does each sample have for OCT4?


## Normalization: counts per million

In order to account for differences in sequencing depth among samples (a sample that has been sequenced deeper will also have more gene counts overall, but this does not necessarily reflect the biology), we **normalize the gene counts** to the **total number of reads in a sample** and multiply by **1 million**. Use the cheat sheet to help you write the code. Normalize your count table and answer the following questions:

1. Which sample is the most deeply sequenced and which one is the most shallow? Can you see this from the normalized table?
2. Which gene has the highest expression value among all the samples after normalization?

## Principal Component Analysis

Principal Component Analysis (PCA) is a dimension reduction method. In order to visualise the distribution of our samples in a 2D plot, we need to reduce the count matrix from its high number of dimensions (in this case, the read count value for each gene represents a dimension) to less, out of which we then plot the first two.

1) PCA requires the data to be transposed and scaled to mean 0 and unit variance. Use the cheat sheet to find the correct function and standardize the data.
2) Run the PCA function. What is the shape of the resulting matrix? What do the rows and columns represent? (which are the samples and which are the principal components?)
3) Plot the first two principal components. How many samples have you plotted?
4) Which samples group together? Are there any outliers?

### Gene markers

Now it is time to label your data. Use the cheat sheet to create a heatmap plot showing known marker genes for each possible differentiation stage:
1. Embryonic Stem Cells
2. Mesoderm
3. Cardiomyocyte progenitors
4. Cardiomyocytes
5. Mature fetal heart

Label the stem cell replicates. Then, identify which stage the other two samples belong to according to the gene expression values in your heatmap. Feel free to consult among yourselves and compare heatmaps if you find it difficult to choose between potential stages.

Add your chosen labels to the metadata table (replace the 'Unknown' in column 'stage' with your chosen label) and verify the clustering in a labelled PCA plot.